In [32]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import pandas as pd
from langfuse.callback import CallbackHandler
import os

load_dotenv()

handler = CallbackHandler(os.environ.get("LANGFUSE_PUBLIC_KEY"), os.environ.get("LANGFUSE_SECRET_KEY"))

In [33]:
MODEL = "gpt-3.5-turbo"

OUTPUT_FOLDER = "../generated_GUIs/"

IMAGE_INPUT_FOLDER = "../data/"

CREATION_PROMPT = PromptTemplate(
    input_variables=["summary"],
    template= """create the HTML code for a Mobilepage, 
    with a bit of styling, that represents following functionality: 
    {summary}."""
)

SUMMARY_PROMPT = PromptTemplate(
    input_variables=["summaries"],
    template="""Given a collection of Screen summaries, which describe the same screen:
        {summaries}.
        
        Extract the provided information of the screen and formulate a new description."""
)

S2W = pd.read_csv("../data/Screen2Words.csv")
S2W = S2W.drop('Unnamed: 0', axis=1)

In [23]:
#function for Creating a GUI based on a given description
def create_gui(id, summary, prompt=CREATION_PROMPT, temp=0):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp)

    chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])

    try: 
        response = chain.run(summary=summary, callbacks=[handler])
    except Exception as e:
        print(e)
    
    print(response)
    with open(OUTPUT_FOLDER + f"{id}.html", 'w') as f:
        f.write(response)


In [34]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser

#Sequential chain, of summarizing the Screen2Words Summaries into one and creating a GUI
def create_gui_S2W(id, summaries, temp=0.5):
    llm = ChatOpenAI(model_name=MODEL, temperature=temp)
    
    summary_chain = SUMMARY_PROMPT | llm | StrOutputParser()
    creation_chain = CREATION_PROMPT | llm | StrOutputParser()
    
    chain = {"summary": summary_chain} | RunnablePassthrough.assign(creation=creation_chain)
    
    summaries_str = "\n".join([f"Summary {i+1}: {summary}" for i, summary in enumerate(summaries)])
    
    try: 
        response = chain.invoke({"summaries": summaries_str}, config={"callbacks":[handler]})
    except Exception as e:
        print(e)
    
    print(response)
    with open(OUTPUT_FOLDER + f"{id}_S2W.html", 'w') as f:
        f.write(response.get('creation'))
    return response

In [35]:
descriptions = S2W[S2W["screenId"]==1264]["summary"].values

response = create_gui_S2W(1264, descriptions)

{'summary': 'The screen is a fitness app page that offers various workout options to set. It also includes a timer and different features related to exercise. Additionally, there is a pop-up providing information about body workouts.', 'creation': '<!DOCTYPE html>\n<html>\n<head>\n<style>\nbody {\n  font-family: Arial, sans-serif;\n  background-color: #f2f2f2;\n}\n\n.container {\n  max-width: 600px;\n  margin: 0 auto;\n  padding: 20px;\n  background-color: white;\n  border-radius: 10px;\n  box-shadow: 0 0 10px rgba(0,0,0,0.1);\n}\n\nh1 {\n  text-align: center;\n  color: #333333;\n}\n\n.workout-options {\n  display: flex;\n  flex-wrap: wrap;\n  justify-content: space-between;\n  margin-top: 20px;\n}\n\n.workout-card {\n  width: 45%;\n  padding: 10px;\n  background-color: #f9f9f9;\n  border-radius: 5px;\n  box-shadow: 0 0 5px rgba(0,0,0,0.1);\n  margin-bottom: 10px;\n}\n\n.workout-card:hover {\n  background-color: #e9e9e9;\n}\n\n.timer {\n  text-align: center;\n  margin-top: 20px;\n}\n\n

In [36]:
S2W[S2W["screenId"]==1264]["summary"].values

array(['page displaying different workout options to set',
       'page displaying to start exercise',
       'page displaying to start workout',
       'pop-up displaying about body work outs',
       'timer and different features of a fitness app are displaying'],
      dtype=object)

In [14]:
DESCRIPTION = "This is a flight booking screen from a travel app where users can search for flights, specifying departure and arrival cities, travel dates, number of passengers, and class of service. It also offers options for round trip or one-way flights and displays a section for recent searches"

create_gui(377, DESCRIPTION)

<!DOCTYPE html>
<html>
<head>
    <title>Flight Booking</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f2f2f2;
        }
        
        .container {
            max-width: 600px;
            margin: 0 auto;
            padding: 20px;
            background-color: #fff;
            border-radius: 5px;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
        }
        
        h1 {
            text-align: center;
            margin-bottom: 20px;
        }
        
        label {
            display: block;
            margin-bottom: 10px;
            font-weight: bold;
        }
        
        input[type="text"],
        select {
            width: 100%;
            padding: 10px;
            border: 1px solid #ccc;
            border-radius: 4px;
            box-sizing: border-box;
            margin-bottom: 20px;
        }
        
        .search-button {
            width: 100%;
            padding: 10px;
      